In [6]:
# 导入必要的库
from datasets import load_dataset
import pandas as pd
from typing import Dict, List
import os
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"

# 加载 HuggingFace 数据集并处理
def process_dataset(dataset_name: str, data_source: str, key: str = "proof") -> pd.DataFrame:
    # 加载数据集
    dataset = load_dataset("json",data_files=dataset_name)
    
    # 假设使用 train 分割，你可以根据需要修改
    data = dataset["train"]
    # data=dataset["validation"]
    
    # 初始化结果列表
    results = []
    
    # 遍历数据集
    for item in data:
        # 获取 proof 字段
        formal_statement=item["formal_statement"]
        proof = item["formal_statement"] + item["proof_code"]
        result=item["result"]
        if not result["sorries"] and not result["errors"]:
            continue
        # 构造 prompt
        prompt = [{
            "content": f"""Fix this proof in Lean4 code:

{proof}

The error messsege is:

{result}

Please fix this Lean4 proof above step by step. After careful reasoning, please provide your fixed Lean4 proof between ```lean and ```.""",
            "role": "user"
        }]
        
        # 构造 reward_model
        reward_model = {
            "ground_truth": {
                "formal_statement": formal_statement
            }
        }
        
        # 构造 extra_info
        extra_info = {k: v for k, v in item.items()}
        
        # 添加到结果
        results.append({
            "data_source": data_source,
            "prompt": prompt,
            "ability": "math",
            "reward_model": reward_model,
            "extra_info": extra_info
        })
    
    # 转换为 DataFrame
    df = pd.DataFrame(results)
    return df

# 示例用法
dataset_name = "/AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/verification_log.jsonl"  # 替换为实际的 HuggingFace 数据集名称
data_source = "lean_fix_wkbk_test"    # 由用户指定
df = process_dataset(dataset_name, data_source)

# 显示前几行


In [ ]:
len(df)

29641

In [3]:
pd.set_option('display.max_colwidth', None)
print(df.head())

NameError: name 'df' is not defined

In [ ]:
def split_and_save_dataset(df: pd.DataFrame, output_dir: str, test_size: int = 16):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 随机打乱数据
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 拆分为测试集和训练集
    test_df = df_shuffled.iloc[:test_size]
    train_df = df_shuffled.iloc[test_size:]
    
    # 保存为 parquet 文件
    test_path = os.path.join(output_dir, "test.parquet")
    train_path = os.path.join(output_dir, "train.parquet")
    
    test_df.to_parquet(test_path, index=False)
    train_df.to_parquet(train_path, index=False)
    
    print(f"Test set saved to: {test_path} ({len(test_df)} rows)")
    print(f"Train set saved to: {train_path} ({len(train_df)} rows)")

# 示例用法

output_dir = "/AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/minif2f"  # 替换为指定的输出路径

# 处理数据集

# 拆分并保存
split_and_save_dataset(df, output_dir, test_size=16)

Test set saved to: /AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/leanwkbk/test.parquet (16 rows)
Train set saved to: /AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/leanwkbk/train.parquet (29625 rows)


In [ ]:
def save_dataset(df: pd.DataFrame, output_dir: str, test_size: int = 16):
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 随机打乱数据
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 拆分为测试集和训练集
    test_df = df_shuffled
    
    # 保存为 parquet 文件
    test_path = os.path.join(output_dir, "test.parquet")

    
    test_df.to_parquet(test_path, index=False)

    
    print(f"Test set saved to: {test_path} ({len(test_df)} rows)")


# 示例用法

output_dir = "/AI4M/users/qzh/lean_test/Agent/Temp/LeanRL/new_verl/verl/data/minif2f"  # 替换为指定的输出路径

# 处理数据集

# 拆分并保存
save_dataset(df, output_dir, test_size=16)